In [3]:
import pandas as pd
from MyCreds.mycreds import IowaData
import pymssql
import h3
from sodapy import Socrata # < Unmaintained as of August 31, 2022, just in time for this project
import numpy as np
from datetime import datetime, timedelta

# Config Globals
KEY = IowaData.api_key
SECRET = IowaData.api_key_secret
TOKEN = IowaData.app_token

liquor_sales_data_id = "m3tr-qhgy"

In [3]:
socrata_df.dtypes

invoice_line_no        object
date                   object
store                  object
name                   object
address                object
city                   object
zipcode                object
store_location         object
county_number          object
county                 object
category               object
category_name          object
vendor_no              object
vendor_name            object
itemno                 object
im_desc                object
pack                   object
bottle_volume_ml       object
state_bottle_cost      object
state_bottle_retail    object
sale_bottles           object
sale_dollars           object
sale_liters            object
sale_gallons           object
dtype: object

In [2]:
client = Socrata("data.iowa.gov", app_token=TOKEN)
results = client.get_all("m3tr-qhgy", where="(date between '2022-01-01' and '2022-01-31')")
socrata_df = pd.DataFrame.from_records(results)
socrata_df

,invoice_line_no,date,store,name,address,city,zipcode,store_location,county_number,county,...,itemno,im_desc,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,sale_bottles,sale_dollars,sale_liters,sale_gallons
0,INV-43468500052,2022-01-02T00:00:00.000,2190,"Central City Liquor, Inc.",1460 2ND AVE,Des Moines,50314,"{'type': 'Point', 'coordinates': [-93.619788, ...",77,POLK,...,5362,John Barr Reserve,12,750,11.67,17.51,3,52.53,2.25,0.59
1,INV-43468400001,2022-01-02T00:00:00.000,2536,HY-VEE / PLEASANT HILL,4815 Maple Drive,Pleasant Hill,50317,"{'type': 'Point', 'coordinates': [-93.519491, ...",77,Polk,...,34001,Absolut Swedish Vodka 80prf Mini,10,50,7.92,11.88,10,118.80,0.50,0.13
2,INV-43468400006,2022-01-02T00:00:00.000,2536,HY-VEE / PLEASANT HILL,4815 Maple Drive,Pleasant Hill,50317,"{'type': 'Point', 'coordinates': [-93.519491, ...",77,Polk,...,39825,Western Son Blueberry,12,750,9.01,13.52,12,162.24,9.00,2.37
3,INV-43468400011,2022-01-02T00:00:00.000,2536,HY-VEE / PLEASANT HILL,4815 Maple Drive,Pleasant Hill,50317,"{'type': 'Point', 'coordinates': [-93.519491, ...",77,Polk,...,25616,Seagrams 7 Crown PET Flask,12,750,7.50,11.25,12,135.00,9.00,2.37
4,INV-43468500039,2022-01-02T00:00:00.000,2190,"Central City Liquor, Inc.",1460 2ND AVE,Des Moines,50314,"{'type': 'Point', 'coordinates': [-93.619788, ...",77,POLK,...,88533,Mi Campo Reposado,12,750,12.40,18.60,2,37.20,1.50,0.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182590,INV-44254200005,2022-01-31T00:00:00.000,3773,Benz Distributing,501 7th Ave SE,Cedar Rapids,52401,"{'type': 'Point', 'coordinates': [-91.659875, ...",57,LINN,...,86390,Montezuma Triple Sec,12,1000,2.13,3.20,48,153.60,48.00,12.68
182591,INV-44267400054,2022-01-31T00:00:00.000,2633,Hy-Vee #3 / BDI / Des Moines,3221 SE 14th St,Des Moines,50320,"{'type': 'Point', 'coordinates': [-93.596756, ...",77,POLK,...,5347,Johnnie Walker Red,12,1000,17.10,25.65,12,307.80,12.00,3.17
182592,INV-44270400014,2022-01-31T00:00:00.000,2200,Sac Liquor Store,619 E Main St,Sac City,50583,"{'type': 'Point', 'coordinates': [-94.974011, ...",81,SAC,...,10834,Crown Royal Black,24,375,8.25,12.38,4,49.52,1.50,0.39
182593,INV-44286800002,2022-01-31T00:00:00.000,5597,Liquor Tobacco & Gas,1401 Central Ave,Dubuque,52001,"{'type': 'Point', 'coordinates': [-90.66783600...",31,DUBUQUE,...,89626,Patron Reposado,12,750,29.00,43.50,12,522.00,9.00,2.37


In [8]:
with open('etl_log.txt', 'a') as f:
    f.write(f"{datetime.now()}, {},{}\n")

In [ ]:
date_between = "(date between '2022-01-01' and '2022-01-31')"

In [8]:
def get_last_date_of_month(year, month):
    """Return the last date of the month.

    Args:
        year (int): Year, i.e. 2022
        month (int): Month, i.e. 1 for January

    Returns:
        date (datetime): Last date of the current month
    """

    if month == 12:
        last_date = datetime(year, month, 31)
    else:
        last_date = datetime(year, month + 1, 1) + timedelta(days=-1)

    return last_date.strftime("%Y-%m-%d")

In [9]:
def log(run_start, time_range, successful=1, error='NaN'):
    run_time = datetime.now() - run_start
    with open('etl_log.csv', 'a') as f:
        f.write(f"{datetime.now()}, {run_time}, {time_range[0]}, {time_range[1]}, {successful}, {error}\n")

In [10]:
def clean_dataframe(df, h3_resolution=8):
    coords_df = pd.DataFrame(df['store_location'].to_dict()).T
    coords_df['coordinates'] = np.where(coords_df['coordinates'].isnull(), pd.Series([[]]*len(coords_df)), coords_df['coordinates']) # fill na with empty list
    df = df.merge(pd.DataFrame(coords_df['coordinates'].to_list(), columns=['long', 'lat']), left_index=True, right_index=True)


    def lat_long_to_h3(row):
        return h3.geo_to_h3(lat=row.lat,lng=row.long,resolution = h3_resolution)

    df['hex_id'] = df.apply(lat_long_to_h3, axis=1)
    df['date'] = pd.to_datetime(df['date'])

    int_cols = ['store', 'county_number', 'category', 'vendor_no', 'itemno', 'pack', 'sale_bottles', ]
    float_cols = ['state_bottle_cost', 'state_bottle_retail', 'sale_dollars', 'sale_liters', 'sale_gallons']

    df[int_cols] = df[int_cols].astype('Int64')
    df[float_cols] = df[float_cols].astype(float)
    df.drop('store_location', axis=1, inplace=True)

    return df

In [11]:
def load(df):
    server = IowaData.azure_server
    user = IowaData.azure_uid
    password = IowaData.azure_pwd

    insert_query = f"""INSERT INTO invoices.temptable({', '.join([*df.columns])}) VALUES ({', '.join(['%s' for col in df.columns])})"""
    write_data = tuple(map(tuple, df.fillna(0).values))

    with pymssql.connect(server, user, password, "iowa_liquor") as cnxn:
        with cnxn.cursor() as cur:
            cur.executemany(insert_query, write_data)
            cnxn.commit()

In [ ]:
date_intervals = [[f"{datetime(y, m, 1):%Y-%m-%d}", f"{get_last_date_of_month(y, m)}"] for y in range(2012, datetime.now().year + 1) for m in range(1, 13)]

client = Socrata("data.iowa.gov", app_token=TOKEN)
success = 1

for interval in date_intervals[:1]:
    run_start_time = datetime.now()
    try:
        date_between = f"(date between '{interval[0]}' and '{interval[1]}')"
        results = client.get_all(liquor_sales_data_id, where=date_between)
        socrata_df = pd.DataFrame.from_records(results)
        socrata_df = clean_dataframe(socrata_df)
        load(socrata_df)
        log(run_start_time, interval)

    except Exception as e:
        log(run_start_time, interval, successful=0, error=e)


socrata_df

In [67]:
socrata_df[socrata_df['zipcode'] == '712-2']

,invoice_line_no,date,store,name,address,city,zipcode,county_number,county,category,...,bottle_volume_ml,state_bottle_cost,state_bottle_retail,sale_bottles,sale_dollars,sale_liters,sale_gallons,long,lat,hex_id
10218,S03437000013,2012-01-04,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1022100,...,1000,12.50,18.74,12,224.88,12.0,3.17,-95.602969,41.854728,88260c4ebbfffff
10472,S03437000007,2012-01-04,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1011100,...,1000,7.50,11.24,12,134.88,12.0,3.17,-95.602969,41.854728,88260c4ebbfffff
10908,S03437000002,2012-01-04,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1081600,...,1000,11.34,17.01,12,204.12,12.0,3.17,-95.602969,41.854728,88260c4ebbfffff
11173,S03437000004,2012-01-04,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1012100,...,1750,30.50,45.74,6,274.44,10.5,2.77,-95.602969,41.854728,88260c4ebbfffff
12485,S03437000006,2012-01-04,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1012100,...,1750,10.45,15.67,12,188.04,21.0,5.55,-95.602969,41.854728,88260c4ebbfffff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118694,S03758800015,2012-01-25,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1081315,...,750,6.94,10.41,2,20.82,1.5,0.40,-95.602969,41.854728,88260c4ebbfffff
119354,S03758800003,2012-01-25,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1062300,...,1000,9.77,14.66,12,175.92,12.0,3.17,-95.602969,41.854728,88260c4ebbfffff
119357,S03758800007,2012-01-25,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1031080,...,1000,3.97,5.95,12,71.40,12.0,3.17,-95.602969,41.854728,88260c4ebbfffff
119430,S03758800011,2012-01-25,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1011200,...,1000,11.54,17.31,12,207.72,12.0,3.17,-95.602969,41.854728,88260c4ebbfffff


In [54]:
socrata_df[socrata_df['zipcode'] == '712-2']

,invoice_line_no,date,store,name,address,city,zipcode,county_number,county,category,...,im_desc,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,sale_bottles,sale_dollars,sale_liters,sale_gallons,store_location
10218,S03437000013,2012-01-04T00:00:00.000,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1022100,...,Jose Cuervo Especial Reposado Tequila,12,1000,12.50,18.74,12,224.88,12.00,3.17,"{'type': 'Point', 'coordinates': [-95.602969, ..."
10472,S03437000007,2012-01-04T00:00:00.000,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1011100,...,Seagrams 7 Crown Bl Whiskey,12,1000,7.50,11.24,12,134.88,12.00,3.17,"{'type': 'Point', 'coordinates': [-95.602969, ..."
10908,S03437000002,2012-01-04T00:00:00.000,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1081600,...,Southern Comfort,12,1000,11.34,17.01,12,204.12,12.00,3.17,"{'type': 'Point', 'coordinates': [-95.602969, ..."
11173,S03437000004,2012-01-04T00:00:00.000,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1012100,...,Crown Royal Canadian Whisky,6,1750,30.50,45.74,6,274.44,10.50,2.77,"{'type': 'Point', 'coordinates': [-95.602969, ..."
12485,S03437000006,2012-01-04T00:00:00.000,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1012100,...,Black Velvet,6,1750,10.45,15.67,12,188.04,21.00,5.55,"{'type': 'Point', 'coordinates': [-95.602969, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119542,S03758800007,2012-01-25T00:00:00.000,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1031080,...,Barton Vodka,12,1000,3.97,5.95,12,71.40,12.00,3.17,"{'type': 'Point', 'coordinates': [-95.602969, ..."
119615,S03758800011,2012-01-25T00:00:00.000,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1011200,...,Jim Beam,12,1000,11.54,17.31,12,207.72,12.00,3.17,"{'type': 'Point', 'coordinates': [-95.602969, ..."
120027,S03758800003,2012-01-25T00:00:00.000,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1062300,...,Malibu Coconut Rum,12,1000,9.77,14.66,12,175.92,12.00,3.17,"{'type': 'Point', 'coordinates': [-95.602969, ..."
120030,S03758800007,2012-01-25T00:00:00.000,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1031080,...,Barton Vodka,12,1000,3.97,5.95,12,71.40,12.00,3.17,"{'type': 'Point', 'coordinates': [-95.602969, ..."


In [42]:
pd.read_csv('etl_log.csv')

,time_stamp,run_time_min,start_date,end_date,successful,error
0,2022-09-09 22:28:50.609779,0:00:55.578335,2012-01-01,2012-01-31,1,
1,2022-09-09 22:30:50.809604,0:00:57.416926,2012-01-01,2012-01-31,1,NaN


In [ ]:
# example

# insert_query = """INSERT INTO dbo.temptable(CHECK_TIME, DEVICE, METRIC, VALUE, TOWER, LOCATION, ANOMALY, ANOMALY_SCORE, ANOMALY_SEVERITY)
#             VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"""
# write_data = tuple(map(tuple, data_frame.values))
# cursor.executemany(insert_query, write_data)
# con.commit()
# cursor.close()
# con.close()


server = IowaData.azure_server
user = IowaData.azure_uid
password = IowaData.azure_pwd

insert_query = f"""INSERT INTO invoices.temptable({', '.join([*socrata_df.columns])}) VALUES ({', '.join(['%s' for col in socrata_df.columns])})"""
write_data = tuple(map(tuple, socrata_df.fillna(0).values))

with pymssql.connect(server, user, password, "iowa_liquor") as cnxn:
    with cnxn.cursor() as cur:
        cur.executemany(insert_query, write_data)
        cnxn.commit()

In [7]:
socrata_df

NameError: name 'socrata_df' is not defined

In [73]:
', '.join(['%s' for col in socrata_df.columns])

'%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s'

In [74]:
len(['%s' for col in socrata_df.columns])

27

In [75]:
len(socrata_df.columns)

27

In [87]:
insert_query = f"""INSERT INTO dbo.temptable({', '.join([*socrata_df.columns])})
            VALUES ({', '.join(['%s' for col in socrata_df.columns])})"""
insert_query


'INSERT INTO dbo.temptable(invoice_line_no, date, store, name, address, city, zipcode, county_number, county, category, category_name, vendor_no, vendor_name, itemno, im_desc, pack, bottle_volume_ml, state_bottle_cost, state_bottle_retail, sale_bottles, sale_dollars, sale_liters, sale_gallons, store_location, long, lat, hex_id)\n            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'

In [9]:
f"({', '.join([*socrata_df.columns])})"

'(invoice_line_no, date, store, name, address, city, zipcode, county_number, county, category, category_name, vendor_no, vendor_name, itemno, im_desc, pack, bottle_volume_ml, state_bottle_cost, state_bottle_retail, sale_bottles, sale_dollars, sale_liters, sale_gallons, long, lat, hex_id)'

In [83]:
(*socrata_df.columns)

SyntaxError: can't use starred expression here (988345262.py, line 1)

In [32]:
write_data = tuple(map(tuple, socrata_df.values))
write_data

(('S03407900038',
  Timestamp('2012-01-03 00:00:00'),
  2549,
  'Hy-Vee Food Store / Indianola',
  '910 N JEFFERSON',
  'INDIANOLA',
  '50125',
  91,
  'Warren',
  1031080,
  'VODKA 80 PROOF',
  260,
  'Diageo Americas',
  37418,
  'Popov Vodka 80 Prf',
  6,
  '1750',
  8.49,
  12.74,
  12,
  152.88,
  21.0,
  5.55,
  nan,
  nan,
  '0'),
 ('S03411000067',
  Timestamp('2012-01-03 00:00:00'),
  2625,
  'Hy-Vee Wine and Spirits #2',
  '3301 W KIMBERLY RD',
  'DAVENPORT',
  '52804',
  82,
  'Scott',
  1011200,
  'STRAIGHT BOURBON WHISKIES',
  65,
  'Jim Beam Brands',
  27544,
  'Red Stag By Jim Beam',
  12,
  '750',
  9.97,
  14.95,
  12,
  179.4,
  9.0,
  2.38,
  nan,
  nan,
  '0'),
 ('S03411000030',
  Timestamp('2012-01-03 00:00:00'),
  2625,
  'Hy-Vee Wine and Spirits #2',
  '3301 W KIMBERLY RD',
  'DAVENPORT',
  '52804',
  82,
  'Scott',
  1062310,
  'SPICED RUM',
  260,
  'Diageo Americas',
  43337,
  'Captain Morgan Spiced Rum',
  12,
  '1000',
  10.99,
  16.49,
  12,
  197.88,
  12.

In [25]:
insert_query = f"""INSERT INTO invoices.temptable({', '.join([*socrata_df.columns])}) VALUES ({', '.join(['%s' for col in socrata_df.columns])})"""
insert_query

'INSERT INTO invoices.temptable(invoice_line_no, date, store, name, address, city, zipcode, county_number, county, category, category_name, vendor_no, vendor_name, itemno, im_desc, pack, bottle_volume_ml, state_bottle_cost, state_bottle_retail, sale_bottles, sale_dollars, sale_liters, sale_gallons, long, lat, hex_id) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'

In [15]:
[*map(tuple, socrata_df.values)]

[('S03407900038',
  Timestamp('2012-01-03 00:00:00'),
  2549,
  'Hy-Vee Food Store / Indianola',
  '910 N JEFFERSON',
  'INDIANOLA',
  '50125',
  91,
  'Warren',
  1031080,
  'VODKA 80 PROOF',
  260,
  'Diageo Americas',
  37418,
  'Popov Vodka 80 Prf',
  6,
  '1750',
  8.49,
  12.74,
  12,
  152.88,
  21.0,
  5.55,
  nan,
  nan,
  '0'),
 ('S03411000067',
  Timestamp('2012-01-03 00:00:00'),
  2625,
  'Hy-Vee Wine and Spirits #2',
  '3301 W KIMBERLY RD',
  'DAVENPORT',
  '52804',
  82,
  'Scott',
  1011200,
  'STRAIGHT BOURBON WHISKIES',
  65,
  'Jim Beam Brands',
  27544,
  'Red Stag By Jim Beam',
  12,
  '750',
  9.97,
  14.95,
  12,
  179.4,
  9.0,
  2.38,
  nan,
  nan,
  '0'),
 ('S03411000030',
  Timestamp('2012-01-03 00:00:00'),
  2625,
  'Hy-Vee Wine and Spirits #2',
  '3301 W KIMBERLY RD',
  'DAVENPORT',
  '52804',
  82,
  'Scott',
  1062310,
  'SPICED RUM',
  260,
  'Diageo Americas',
  43337,
  'Captain Morgan Spiced Rum',
  12,
  '1000',
  10.99,
  16.49,
  12,
  197.88,
  12.

In [18]:
socrata_df.to_records(index=False)

rec.array([('S03407900038', '2012-01-03T00:00:00.000000000', 2549, 'Hy-Vee Food Store / Indianola', '910 N JEFFERSON', 'INDIANOLA', '50125', 91, 'Warren', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37418, 'Popov Vodka 80 Prf', 6, '1750',  8.49, 12.74, 12, 152.88, 21.  , 5.55,        nan,       nan, '0'),
           ('S03411000067', '2012-01-03T00:00:00.000000000', 2625, 'Hy-Vee Wine and Spirits #2', '3301 W KIMBERLY RD', 'DAVENPORT', '52804', 82, 'Scott', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 27544, 'Red Stag By Jim Beam', 12, '750',  9.97, 14.95, 12, 179.4 ,  9.  , 2.38,        nan,       nan, '0'),
           ('S03411000030', '2012-01-03T00:00:00.000000000', 2625, 'Hy-Vee Wine and Spirits #2', '3301 W KIMBERLY RD', 'DAVENPORT', '52804', 82, 'Scott', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43337, 'Captain Morgan Spiced Rum', 12, '1000', 10.99, 16.49, 12, 197.88, 12.  , 3.17,        nan,       nan, '0'),
           ...,
           ('S03846900020', 

In [24]:
for col in socrata_df.columns:
    print(col)

invoice_line_no
date
store
name
address
city
zipcode
county_number
county
category
category_name
vendor_no
vendor_name
itemno
im_desc
pack
bottle_volume_ml
state_bottle_cost
state_bottle_retail
sale_bottles
sale_dollars
sale_liters
sale_gallons
long
lat
hex_id


In [46]:
socrata_df['hex_id'].str.len().max()

15

In [5]:
server = IowaData.azure_server
user = IowaData.azure_uid
password = IowaData.azure_pwd

st = datetime.now()
sql = "select * from invoices.temptable"

with pymssql.connect(server, user, password, "iowa_liquor") as cnxn:
    df_sql = pd.read_sql(sql, cnxn)
et = datetime.now()
print(f"Run time: {et - st}")
df_sql

Run time: 0:00:00.888834


/Users/michaelmcmanus/.conda/envs/iowa/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,invoice_line_no,date,store,name,address,city,zipcode,county_number,county,category,...,bottle_volume_ml,state_bottle_cost,state_bottle_retail,sale_bottles,sale_dollars,sale_liters,sale_gallons,long,lat,hex_id
